In [ ]:
import torch as th
from torch.nn import functional as th_f
import torchaudio as th_audio
from torchaudio import functional as th_audio_f
import matplotlib.pyplot as plt

from music_diffusion.data import simpson

# Analogique vs. numérique

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Pcm.svg/1280px-Pcm.svg.png" width="40%" height="40%"/>

- Sample rate (taux d'échantillonage) : fréquence à laquelle le signal analogique est mesuré / enregistré.
- Bit depth (quantification) : le nombre de valeurs possibles que peut prendre le signal sur un échantillon. (ici $2^4 = 16$ valeurs possible, 4-bits).

On ne peut pas "zoomer" à l'infini (espace mémoire) et on ne veut pas pour autant détruire le signal.

Le CD : 
- échantilloné à 44100Hz donc avec une fréquence maximale de 22050Hz, registre humain 20Hz (grave) à ~20000Hz (aigüe) : perfect
- usuellement quantifié avec 16 ou 32 bits : $2^{16} = 65536$ groupes possibles pour un signal entre [-1; 1]


# Signal / Composition de fréquences

In [ ]:
duree = 5  # seconds
sample_rate = 44100  # 44100 Hz

In [ ]:
ticks = th.linspace(0, duree, steps=duree * sample_rate)
print(ticks.size(), duree * sample_rate)
print(ticks.min(), ticks.max())

In [ ]:
la_440 = th.cos(440 * 2 * th.pi * ticks)
print(la_440.size(), la_440.min(), la_440.max())

In [ ]:
time_limit_seconds = 1. / 100.
time_limit_ticks = int(sample_rate * time_limit_seconds)

In [ ]:
plt.plot(ticks[:time_limit_ticks], la_440[:time_limit_ticks])

In [ ]:
th_audio.save("la_440.wav", la_440[None, :], sample_rate)

In [ ]:
la_880 = th.cos(880 * 2 * th.pi * ticks)

In [ ]:
plt.plot(ticks[:time_limit_ticks], la_440[:time_limit_ticks], color="r")
plt.plot(ticks[:time_limit_ticks], la_880[:time_limit_ticks], color="g")

In [ ]:
th_audio.save("la_880.wav", la_880[None, :], sample_rate)

In [ ]:
la_440_la_880 = la_440 + la_880

In [ ]:
plt.plot(ticks[:time_limit_ticks], la_440_la_880[:time_limit_ticks])

In [ ]:
th_audio.save("la_440_la_880.wav", la_440_la_880[None, :], sample_rate)

In [ ]:
do_3 = th.cos(261.6 * 2. * th.pi * ticks)
re_diese_3 = th.cos(311.1 * 2 * th.pi * ticks)
mi_3 = th.cos(329.6 * 2. * th.pi * ticks)
sol_3 = th.cos(392.0 * 2. * th.pi * ticks)
do_4 = th.cos(523.3 * 2. * th.pi * ticks)

In [ ]:
majeur = do_3 + mi_3 + sol_3 + do_4
mineur = do_3 + re_diese_3 + sol_3 + do_4

th_audio.save("majeur.wav", majeur[None, :], sample_rate)
th_audio.save("mineur.wav", mineur[None, :], sample_rate)

# Avec un vrai audio

In [ ]:
raw_audio, sample_rate = th_audio.load("./resources/mystere.flac")

In [ ]:
# stereo
print(raw_audio.size())
print(raw_audio.dtype)
print(raw_audio.min(), raw_audio.max())

In [ ]:
print(f"Sample rate {sample_rate}")

In [ ]:
print(f"Durée : {raw_audio.size(1) / sample_rate / 60.} minutes")

In [ ]:
raw_audio_mono = raw_audio.mean(dim=0)

In [ ]:
plt.plot(ticks[:100], raw_audio_mono[400:500])

In [ ]:
plt.plot(ticks[:1000], raw_audio_mono[400:1400])

Pas très représentatif de la musique (notes, timbres, etc.) ?

Solution : passer dans le domaine des fréquences avec une transformée de Fourier par exemple :

![](https://www.nti-audio.com/portals/0/pic/news/FFT-Time-Frequency-View-540.png)

Un exemple de signal dans le domaine des fréquences (spectrogramme) :

![](https://www.numerical-tours.com/matlab/audio_1_processing/index_09.png)

Nous allons utiliser la STFT (Short Time Fourier Transform) :

![](https://www.researchgate.net/publication/346243843/figure/fig1/AS:961807523000322@1606324191138/Short-time-Fourier-transform-STFT-overview.png)

In [ ]:
n_per_seg = 1024 # Donc 512 "paquets" de fréquences
stride = 256 # On décale la fenêtre de la STFT de 256 ticks (à 44100 Hz !)

In [ ]:
complex_values = th_audio_f.spectrogram(
        raw_audio_mono,
        pad=0,
        window=th.hann_window(n_per_seg),
        n_fft=n_per_seg,
        hop_length=stride,
        win_length=n_per_seg,
        power=None,
        normalized=True,
    )

In [ ]:
print(complex_values.size())

In [ ]:
print(complex_values.dtype)

Le dernier paquet de fréquence (la 513e) ne sert à rien : en effet sa fréquence oscille plus vite que le taux d'échantillonage (44100 Hz)

In [ ]:
complex_values = complex_values[:-1, :]

In [ ]:
print(complex_values.size())

In [ ]:
offset = 10000

In [ ]:
plt.matshow(th.real(complex_values[:, offset:offset+512]), cmap="plasma")
plt.matshow(th.imag(complex_values[:, offset:offset+512]), cmap="plasma")

Toujours pas très représentatif...

Pourquoi ne pas utiliser la représentation exponetielle des complexes ?

![](https://mathonweb.com/help_ebook/complex/complex24.gif)

In [ ]:
magnitude = th.abs(complex_values)
phase = th.angle(complex_values)

![](https://homepages.inf.ed.ac.uk/rbf/CVonline/LOCAL_COPIES/OWENS/LECT4/img21.gif)

In [ ]:
plt.matshow(magnitude[:, offset:offset+512], cmap="plasma")

In [ ]:
plt.matshow(phase[:, offset:offset+512], cmap="plasma")

La phase ressemble plus à du bruit qu'autre chose. Essayons de comprendre pourquoi :

## 1er effet indésirable : le décalage

In [ ]:
shift = 10
sinusoide_441_hz = th.cos(441 * 2. * th.pi * ticks)
sinusoide_441_hz_shifted = th.cos(441 * 2. * th.pi * ticks - th.pi / 4.)

stft_window_100 = 100

In [ ]:
plt.plot(ticks[:time_limit_ticks], sinusoide_441_hz[:time_limit_ticks], color="r")
plt.plot(ticks[:time_limit_ticks], sinusoide_441_hz_shifted[:time_limit_ticks], color="g")

curr_window_start = 0
while curr_window_start < time_limit_ticks:
    plt.axvline(x=ticks[curr_window_start], color="cyan")
    curr_window_start += stft_window_100

Si le décalage est constant sur toute la musique, ça passe encore !

mais...

## 2e effet indésirable : le décalage du décalage

Car oui, le début de la fenêtre de STFT et une fréquence peuvent être décalés.

In [ ]:
sinusoid_500_hz_shifted = th.cos(500. * 2. * th.pi * ticks - th.pi / 4.)

In [ ]:
plt.plot(ticks[:time_limit_ticks], sinusoid_500_hz_shifted[:time_limit_ticks], color="r")
curr_window_start = 0
while curr_window_start < time_limit_ticks:
    plt.axvline(x=ticks[curr_window_start], color="cyan")
    curr_window_start += stft_window_100

## Phase instantanée

Approches retenue : GANSynth, le GAN de magenta qui synthtise des notes / timbres d'instruments, ex: un do de trompette.

Les auteurs du papier proposent une transformation pour rendre la phase plus "facile". Par plus facile j'entends de faire apparaitre des motifs qui seront apprenables par un algorithme de génération d'images.

![](https://media.arxiv-vanity.com/render-output/6223267/GANSynth_figs_motivation.png)

Plus qu'à coder la transformation !

### 1. Unwrap

Des gens l'ont déjà fait et j'avais la flemme de chercher :p

In [ ]:
def diff(x: th.Tensor) -> th.Tensor:
    return th_f.pad(x[:, 1:] - x[:, :-1], (1, 0, 0, 0), "constant", 0)


# https://discuss.pytorch.org/t/np-unwrap-function-in-pytorch/34688/2
def unwrap(phi: th.Tensor) -> th.Tensor:
    d_phi = diff(phi)
    d_phi_m = ((d_phi + th.pi) % (2 * th.pi)) - th.pi
    d_phi_m[(d_phi_m == -th.pi) & (d_phi > 0)] = th.pi
    phi_adj = d_phi_m - d_phi
    phi_adj[d_phi.abs() < th.pi] = 0
    return phi + phi_adj.cumsum(1)

In [ ]:
unwrapped_phase = unwrap(phase)

In [ ]:
plt.matshow(unwrapped_phase[:, :512], cmap="plasma")

In [ ]:
plt.matshow(unwrapped_phase[:, offset:offset+512], cmap="plasma")

### 2. "Taux d'accroissement" / "dérivée" / aka phase instantanée

L'idée : représenter des combien (en temps) la phase d'une fréquence se décale par rapport au début d'une fenêtre de STFT.

C'est parti pour calculer un gradient !

In [ ]:
# première phase à 0 pour chaque fréquence
derived_phase_tmp = th_f.pad(unwrapped_phase, (1, 0, 0, 0), "constant", 0.0)
# le delta est de 1 tick, on applique le gradient sur l'axe du temps
derived_phase = th.gradient(derived_phase_tmp, dim=1, spacing=1.0, edge_order=1)[0]

In [ ]:
magnitude = th_f.pad(magnitude, (1, 0, 0, 0), "constant", 0.0)

In [ ]:
plt.matshow(derived_phase[:, :512], cmap="plasma")

In [ ]:
plt.matshow(derived_phase[:, offset:offset+512], cmap="plasma")

## Dernière étape : echelle de bark sur la magnitude

L'humain perçoit / discerne mieux les fréquences dans le registre medium (environ < 4000 Hz), traduisons le dans les magnitudes.
L'idée : partir sur un modèle psychoacoustique type echelle de Bark ou de Mel pour mieux faire ressortir les fréquences aigües.

Ici, echelle de Bark :

![](https://upload.wikimedia.org/wikipedia/commons/2/20/Bark_scale.png)

In [ ]:
min_hz = 20.0
max_hz = 44100 // 2

lin_space: th.Tensor = (
    th.linspace(min_hz, max_hz, magnitude.size(0)) / 600.0
)
bark_scale = 6.0 * th.arcsinh(lin_space)[:, None]
bark_scale = bark_scale / bark_scale[-1, :]

In [ ]:
scaled_magnitude = magnitude * bark_scale

In [ ]:
plt.matshow(scaled_magnitude[:, offset:offset+512], cmap="plasma")

## Images à 2 couleurs

La magnitude et la phase !

In [ ]:
fig, (magn_ax, phase_ax) = plt.subplots(1, 2)

magn_ax.matshow(scaled_magnitude[:, offset:offset + 512], cmap="plasma")
magn_ax.set_title(f"Magnitude")

# Plot phase

phase_ax.matshow(derived_phase[:, offset:offset + 512], cmap="plasma")
phase_ax.set_title(f"Phase")

fig.savefig(f"magn_phase.png")

plt.show()

plt.close()

## Transformation inverse

Une image ça ne s'écoute pas !

In [ ]:
# inverse de la mise à l'échelle Bark
descaled_magnitude = scaled_magnitude / bark_scale

# inverse du gradient
reconstructed_phase = simpson(th.zeros(derived_phase.size()[0], 1), derived_phase, 1, 1.0)

# inverse de l'unwrap
reconstructed_phase = reconstructed_phase % (2 * th.pi)

# passage partie réelle et imaginaire
real = descaled_magnitude * th.cos(reconstructed_phase)
imaginary = descaled_magnitude * th.sin(reconstructed_phase)

# ajout de la fréquence Nyquist
real_res = th_f.pad(real, (0, 0, 0, 1), "constant", 0)
imaginary_res = th_f.pad(imaginary, (0, 0, 0, 1), "constant", 0)

# création du tensor de complexes
z = real_res + imaginary_res * 1j

# STFT inverse
raw_audio = th_audio_f.inverse_spectrogram(
    z,
    length=None,
    pad=0,
    window=th.hann_window(n_per_seg),
    n_fft=n_per_seg,
    hop_length=stride,
    win_length=n_per_seg,
    normalized=True,
)

# sauvegarde de l'audio
th_audio.save("reconstructed.wav", raw_audio[None, :], sample_rate)